In [1]:
import os 
%pwd

'c:\\Users\\j_san\\OneDrive\\Desktop\\AIML\\MLOps\\Project\\textSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\j_san\\OneDrive\\Desktop\\AIML\\MLOps\\Project\\textSummarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config



In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

c:\Users\j_san\OneDrive\Desktop\AIML\MLOps\Project\textSummarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))




In [9]:
# Upgrade transformers to ensure compatibility with evaluation_strategy argument
%pip install --upgrade transformers



  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Restart kernel and reload configuration to ensure latest params.yaml is loaded
import importlib
import sys

# Clear any cached modules
if 'src.textSummarizer.utils.common' in sys.modules:
    importlib.reload(sys.modules['src.textSummarizer.utils.common'])

# Re-import and test the configuration
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

# Test reading the params file directly
test_params = read_yaml(PARAMS_FILE_PATH)
print("Current eval_strategy value:", test_params.TrainingArguments.eval_strategy)


[2025-09-28 21:22:35,635: INFO: common: yaml file: params.yaml loaded successfully]
Current eval_strategy value: steps


In [10]:
# Test the configuration manager with the updated params
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
print("Model trainer config created successfully!")
print(f"Eval strategy: {model_trainer_config.eval_strategy}")
print(f"Eval steps: {model_trainer_config.eval_steps}")
print(f"Root dir: {model_trainer_config.root_dir}")


[2025-09-28 21:22:49,706: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-28 21:22:49,709: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-28 21:22:49,710: INFO: common: created directory at: artifacts]
[2025-09-28 21:22:49,712: INFO: common: created directory at: artifacts/model_trainer]
Model trainer config created successfully!
Eval strategy: steps
Eval steps: 500
Root dir: artifacts/model_trainer


In [11]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-09-28 21:23:32,140: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-28 21:23:32,143: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-28 21:23:32,144: INFO: common: created directory at: artifacts]
[2025-09-28 21:23:32,145: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\j_san\AppData\Local\Temp\ipykernel_19588\2273714221.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
c:\Users\j_san\OneDrive\Desktop\AIML\MLOps\Project\textSummarizer\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_mem

Step,Training Loss,Validation Loss


c:\Users\j_san\OneDrive\Desktop\AIML\MLOps\Project\textSummarizer\venv\lib\site-packages\transformers\modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
